In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
import numpy as np
import re
import os, shutil

In [98]:
data_list = []
regex = re.compile('[^a-zA-Z]')
folder = '/content/drive/MyDrive/Study/ML/dataset'

total = len(os.listdir(folder))
part = total//10
count = 0

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    with open(file_path, 'r') as f:
        text = f.read()
        text = [regex.sub('', x.lower()).strip() for x in text.split()]
        data_list.extend(text)
    if count%part==0:
      print(f'Process {count*10//part}% of the text')
    count+=1
dataset =  [x for x in data_list if x != '']

Process 0% of the text
Process 10% of the text
Process 20% of the text
Process 30% of the text
Process 40% of the text
Process 50% of the text
Process 60% of the text
Process 70% of the text
Process 80% of the text
Process 90% of the text
Process 100% of the text


In [109]:
print('Number of samples:', total)
print('Number of words:', len(dataset))

Number of samples: 273
Number of words: 29127


In [99]:
def generateTable(data):
    table = {}
    for i in range(len(data)-1):
        X = data[i]
        Y = data[i+1]

        if X not in table.keys():
            table[X] = {}
            table[X][Y] = 1
        else:
            if Y not in table[X].keys():
                table[X][Y] = 1
            else:
                table[X][Y] += 1
    
    return table

table = generateTable(['dear', 'love', 'i', 'remember', 'you', 'i', 'love', 'you', 'love', 'you'])
print(table)

{'dear': {'love': 1}, 'love': {'i': 1, 'you': 2}, 'i': {'remember': 1, 'love': 1}, 'remember': {'you': 1}, 'you': {'i': 1, 'love': 1}}


In [100]:
def convertFreqIntoProb(table):     
    for word in table.keys():
        s = float(sum(table[word].values()))
        for prob in table[word].keys():
            table[word][prob] = table[word][prob]/s
    return table
 
table = convertFreqIntoProb(table)
print(table)

{'dear': {'love': 1.0}, 'love': {'i': 0.3333333333333333, 'you': 0.6666666666666666}, 'i': {'remember': 0.5, 'love': 0.5}, 'remember': {'you': 1.0}, 'you': {'i': 0.5, 'love': 0.5}}


In [101]:
table = generateTable(dataset)
chain = convertFreqIntoProb(table)

In [102]:
def sample_next(word,chain):
    if word not in chain.keys():
        return "\n"
    following_words = list(chain[word].keys())
    probs = list(chain[word].values())

    mean = np.mean(probs)
    best_next = []
    for var in following_words:
        if chain[word][var] > mean:
            best_next.append(var)

    if len(best_next) == 0:
        best_next = following_words
    
    return np.random.choice(best_next)
 

In [104]:
def generateText(first_word, chain, maxLen=100):

    sentence = [first_word]
    target_word = first_word
    
    for i in range(maxLen):
        next_prediction = sample_next(target_word,chain)
        sentence.append(next_prediction)
        target_word = sentence[-1]
        if target_word == '\n':
            break

    return ' '.join(sentence)

In [106]:
text = generateText("dear",chain,maxLen=50)
print(text)

dear little interest as soon as we will not think about me no more than that was until every now that has lived within me that our separation gives me all your head upon my joy regardless of his misshapen and to you have in one can replace you smile i
